# Curso de Python para Economistas
## Bienvenidos a la clase 5

### Anuncios y temario para hoy

- Pull de GitHub
- Cuestionario dificultad tarea
- Respuestas del cuestionario de los videos
- Dudas de la teoría
- Ejercicio práctico: Analisis de sentimiento de tweets de Joe Biden pre y post elecciones

### Cuestionario dificultad Tarea
https://forms.gle/yCLHV93TFNLd9Ug86

### Respuestas a las preguntas de los videos:
https://docs.google.com/forms/d/e/1FAIpQLSfb7zlEHh3gllD3iUUWwPCMwyQPd0Z04F3XpArSKQzPkYvGHA/viewform?usp=pp_url&entry.1318485631=Respuesta+Satisfactoria+(Successful+responses)&entry.330305262=Una+celda+de+una+tabla&entry.871102831=Tabula&entry.871102831=PyPDF2&entry.871102831=Minecart&entry.2076477619=La+puedo+trabajar+desde+python+con+el+paquete+psycopg2

### Analisis de sentimiento de tweets de Joe Biden pre y post elecciones

In [3]:
#importamos los paquetes a utilizar
import os
import re
import string
import pandas as pd
import numpy as np
import tweepy

import sys
!{sys.executable} -m pip install wordcloud
from wordcloud import WordCloud
!{sys.executable} -m pip install Pillow
from PIL import Image
import matplotlib.pyplot as plt


from textblob import TextBlob
from datetime import datetime
#Si alguien quiere trabajar con alguna cuenta en español podrían instalar 
#sentiment_analysis_spanish y utilizarlo en reemplado de TextBlob:
#!{sys.executable} -m pip install sentiment_analysis_spanish
#from sentiment_analysis_spanish import sentiment_analysis

import nltk
nltk.download('stopwords')
# Si trabajan en español pueden crearse una lista de stopwords con las de este link:
# https://github.com/xiamx/node-nltk-stopwords/blob/master/data/stopwords/spanish
# También usaremos string.punctuation. Si trabajan en español podrian agregarle  
# ¿ y ¡ . En ingles incluye lo siguiente: !"#$%&'()*+, -./:;<=>?@[\]^_`{|}~
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/belenmichel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Para generar las claves vayan a: 
https://developer.twitter.com/en/portal/dashboard

- Luego a Projects & Apps
- Luego Overview
- Luego + Create App

Crear con algun editor de texto (ej. Sublime) un archivo llamado `twitter_keys.txt` dentro de la carpeta `clase5` y guardar las 4 claves, una por línea, en el siguiente orden:
- API key
- API key secret
- Access token
- Access token secret

In [4]:
#Creamos variables que contienen nuestas claves de autenticación con la API
with open("twitter_keys.txt") as tw_k: 
    consumer_key = tw_k.readline().strip()
    consumer_secret = tw_k.readline().strip()
    access_key = tw_k.readline().strip()
    access_secret = tw_k.readline().strip()

In [5]:
#Le pasamos nuestras credenciales de twitter a tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

En este link pueden explorar detalles del metodo user_timeline: https://docs.tweepy.org/en/stable/api.html?highlight=user_timeline#tweepy.API.user_timeline

In [6]:
def get_all_tweets(screen_name, start_date):
    '''
    Esta funcion recibe el nombre de la persona de quien queremos extraer los 
    tweets y devuelve una lista con todos los tweets y sus datos
    Input: 
      screen_name (str): el nombre de la persona en twitter
    Output:
      all_tweets (lista): lista con todos los tweets extraidos
    '''
    # Solicitamos los 200 tweets mas recientes (200 es el maximo permitido en count)
    new_tweets = api.user_timeline(screen_name=screen_name, 
                                   tweet_mode="extended", count=200)
    # Creo una lista para almacenar TODOS los tweets y agrego los recién extraidos
    all_tweets = []
    all_tweets.extend(new_tweets)
    # guardo el id del ultimo tweet extraído 
    oldest = all_tweets[-1].id 
    
    # Extraigo tweets de a 200 hasta que no haya más
    while len(new_tweets) > 0 and all_tweets[-1].created_at > start_date:
        # Solicito 200 tweets mas y los agrego a la lista de 'all_tweets'
        new_tweets = api.user_timeline(screen_name=screen_name, count=200,
                                       tweet_mode="extended", max_id=oldest-1)
        all_tweets.extend(new_tweets)
        # Actualizo el id del ultimo tweet extraído
        oldest = all_tweets[-1].id 
        print("Hasta ahora se han extraido %s tweets" % len(all_tweets))

    return all_tweets


In [7]:
# Extraemos los tweets desde unos días antes de las elecciones del 
# 3 de Noviembre de 2020
date_before_elections = datetime(2020, 10, 16, 0, 0, 0)
all_tweets_biden = get_all_tweets("JoeBiden", date_before_elections)

Hasta ahora se han extraido 400 tweets
Hasta ahora se han extraido 600 tweets
Hasta ahora se han extraido 800 tweets
Hasta ahora se han extraido 1000 tweets
Hasta ahora se han extraido 1200 tweets
Hasta ahora se han extraido 1400 tweets


In [8]:
#Veamos la lista con los primeros 5 objetos de tweepy
all_tweets_biden[:5]

[Status(_api=<tweepy.api.API object at 0x7fa1dded2d50>, _json={'created_at': 'Wed Oct 20 19:02:18 +0000 2021', 'id': 1450900197043757058, 'id_str': '1450900197043757058', 'full_text': 'RT @POTUS: The sacred right to vote is under unrelenting assault across the country — and the Senate needs to take action to protect it. Se…', 'truncated': False, 'display_text_range': [0, 140], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'POTUS', 'name': 'President Biden', 'id': 1349149096909668363, 'id_str': '1349149096909668363', 'indices': [3, 9]}], 'urls': []}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 939091, 'id_str': '939091', 'name': 'Joe Biden', 'screen_name': 'JoeBiden', 'location': 'Washington, DC', 'description': 'Husband to @DrBiden, proud fa

In [ ]:
# Veamos un solo tweet 
all_tweets_biden[8]

In [ ]:
# Veamos el texto de un solo tweet 
all_tweets_biden[8].full_text

#### Ejercicio 1: 
En la siguiente función construyan un loop que extraiga id_str, created_at, full_text, retweeted,  favorite_count, in_reply_to_screen_name de cada tweet y lo guarde en una lista. Esa lista se debe agregar al final de la lista all_tweets_selection para luego construir un df con la lista de listas (las listas internas serán las filas del df). 

In [ ]:
def save_tweets_text(all_tweets, csv_file=None):
    '''
    Esta función guarda los tweets en un data frame y si se especifica un 
    archivo csv tambien se guardaran ahí 
    Input:
        all_tweets (lista): lista con tweets y sus datos
        csv_file ('str'): nombre del archivo csv
    Output:
        df_all_tweets (df): tweets ordenados en una tabla con datos seleccinados
    '''
    all_tweets_selection = []
    
    # Su Código acá:
    
    # Hasta acá su código    
    df_all_tweets = pd.DataFrame(all_tweets_selection)
    df_all_tweets.columns = ['id_str', 'created_at', 'text', 'retweeted',
                            'favorite_count', 'in_reply_to_screen_name']
    if csv_file:
        df_all_tweets.to_csv(csv_file, index=False, encoding="utf-8")

    return df_all_tweets

#### Ejercicio 1b: 
Utilicen la función save_tweets_text para construir una tabla con todos los tweets y guardenlo en una archivo llamado "tweets.csv". Por ultimo visualicen algunas filas de la tabla. 

 #### Ejercicio 1b: 
 Hagan print del texto de algunos tweets y luego piensen:
 
 1) Les parece que podríamos hacer un análisis de sentimiento de estos tweets como están?
 
 2) Porque?
 
 3) Que errores creen que podriamos tener al trabajar con el texto tal como lo descargamos de twitter?
 
 4) Que transformaciones le harían al texto antes de meterlo en el análisis (mencionen al menos 5 cosas)

In [ ]:
# Emoticones contentos
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
 
# Emoticones Tristes
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

# Combinamos emoticones contentos y tristes
emoticons = emoticons_happy.union(emoticons_sad)

In [ ]:
#Emoji patterns
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # simbolos & pictogramas
                           u"\U0001F680-\U0001F6FF"  # transporte & simbolos mapas
                           u"\U0001F1E0-\U0001F1FF"  # banderas (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)


In [ ]:
def clean_tweet(tweet):
    '''
    Esta función limpia el texto de un tweet. Elimina caracteres especificos que
    se utilizan en twitter como los de re-tweets, los links y otros Non-ASCII.
    Devuelve el texto "limpio".
    Input:
        tweet (str): Texto del tweet original
    Output:
        tweet (str): Texto del tweet limpiado
    '''   
    #Saco los links
    tweet = re.sub(r'https.*', '', tweet)
    #Elimino caracteres de re-tweets   
    tweet = re.sub(r'^RT .*:', '', tweet)
    tweet = re.sub(r'@\S+', '', tweet)
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
    #Reemplazo caracteres non-ASCII con espacio
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
    
    return tweet
    

In [ ]:
#Este es un tweet sucio:
df_all_tweets['text'][1]

In [ ]:
#Este es un tweet limpio:
tweet_cleaned = clean_tweet(df_all_tweets.iloc[1]['text'])
tweet_cleaned

#### Análisis de Sentimiento
https://textblob.readthedocs.io/en/dev/api_reference.html#module-textblob.en.sentiments
- Polarity: Negative (-1.0) vs. Positive (1.0)
- Subjectivity: Objective (0.0) vs. Subjective (1.0)

In [ ]:
# Calculamos el sentimiento con el metodo TextBlob
blob = TextBlob(tweet_cleaned)
Sentiment = blob.sentiment
polarity = Sentiment.polarity
subjectivity = Sentiment.subjectivity  
print("La polaridad de este tweet es :", polarity)

#### Ejercicio 2:
Construyamos un loop que nos permita limpiar todos los strings y concatenarlos. Al tenerlos todos concatenados será más facil utilizarlos en un grafico, como por ejemplo una nube de pablabras 

In [ ]:
# Generamos una imagen de nube de palabras 
# wordcloud = WordCloud().generate(all_tweets_cleaned)

wordcloud = WordCloud().generate(all_tweets) #prueben incluir: max_font_size=40
plt.figure(figsize=(12,9))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()


wordcloud = WordCloud(background_color = 'white', stopwords = stopwords.words('english')).generate(all_tweets_cleaned)
plt.figure(figsize=(12,9))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()



In [ ]:
with Image.open('upvote.png') as im:
    im.rotate(45).show()

In [ ]:
# Import image to np.array
mask = np.array(Image.open('upvote.png'))

# Generate wordcloud
wordcloud = WordCloud(width = 3000, height = 2000, background_color='white', 
                      collocations=False, stopwords = stopwords.words('english'), 
                      mask=mask).generate(all_tweets_cleaned)

plt.figure(figsize=(40, 30))
plt.imshow(wordcloud) 
plt.axis("off")

Nota: La imagen del dedo la tomé como idea de este blog https://towardsdatascience.com/simple-wordcloud-in-python-2ae54a9f58e5

#### Ejercicio 3:
Creen una función que limpie texto en general. Elimina emoticones, emojis, palabras vacías (también llamadas stop words en la libreria de nltk), links, puntuaciones, indicaciones de retweets, etc. Esta función dejara el texto limpio y solo con las palabras que pueden agregar mayor valor para el análisis.

In [ ]:
def clean_text(text, is_tweet=False):
    '''
    Esta función limpia el texto del tweet. Elimina emoticones, emojis, palabras
    vacías (también llamadas stop words en la libreria de nltk), links, puntuaciones
    indicaciones de retweets, etc. Para dejar en el texto solo las palabras con 
    mayor contenido.
    Input:
        text (str): Texto original
        is_tweet (bool): si el texto es un tweet este parametro debe setearse a 
                         true para que también se limpie el texto con los 
                         caracteres más especificos de twitter (usa clean_tweet) 
    Output:
        text (str): Texto limpiado
    '''


In [ ]:
df_all_tweets['text'][1]

In [ ]:
clean_text(df_all_tweets['text'][1], is_tweet=True)